In [1]:
from pyspark import SparkConf,SparkContext

In [2]:
conf = SparkConf().setMaster('local').setAppName('weaterdata') 
sc = SparkContext(conf=conf)

In [5]:
def parsedata(line):
    field = line.split(",")
    stationID = field[0]
    entrytype = field[2]
    temperature = float(field[3])*0.1*(9.0/5.0)+32.0
    return(stationID,entrytype,temperature)

In [27]:
lines = sc.textFile('1800.csv')
parselines = lines.map(parsedata)

In [28]:
# filter() - take a func that returns a boolean if true values go to new RDD else discarded 
minTemps= parselines.filter(lambda x:"TMIN" in x[1]) # remove values that don't have TMIN as entrytype

In [29]:
# Creating (stationID and temperature) KEY/VALUE pairs
stationTemps = minTemps.map(lambda x:(x[0],x[2]))

In [30]:
# Min temperature of entire year
minTemps = stationTemps.reduceByKey(lambda x,y:min(x,y))

In [31]:
results = minTemps.collect()

for result in results:
    print (result[0]+"\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F
